## Using Mistral-7B with MAX Engine 🏎️ on CPU

**In this notebook we will walk through an example of using [Mistral-7B](https://huggingface.co/mistralai/Mistral-7B-v0.1) model with MAX Engine 🏎️ on CPU and float32. Check out the [roadmap](https://docs.modular.com/max/roadmap) for quantization and the GPU support.**

The Mistral-7B-v0.1 Large Language Model (LLM) is a pretrained generative text model with 7 billion parameters. Generative text models generate the next token iteratively given a sequence of past tokens representing the input prompt plus already generated response tokens.

Thus the underlying transformer model is invoked in each iteration of this loop until we reach the stopping condition (either the maximum number of generated tokens or a token designated as the end).

**Caveats:**

* **The model size is 28G**. Please make sure you have enough disk space to download the model and for the converted ONNX counterpart as we will use them later in this tutorial.
* **The runtime memory requirment is around 65G** to be able finish the notebook and run all the cells.

First, make sure you have installed `max` as described in the [getting started](https://docs.staging.modular.com/engine/get-started/) as well as `PyTorch` and `transformers` packages

In [ ]:
!python3 -m pip install -q torch --index-url https://download.pytorch.org/whl/cpu
!python3 -m pip install -q transformers onnx

## Vanilla transformers

Let's first see how the model generates a response using the vanilla `transformers`

In [ ]:
%%time

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

PROMPT = "Why did the chicken cross the road?"

hf_path = "mistralai/Mistral-7B-v0.1"
hfmodel = AutoModelForCausalLM.from_pretrained(hf_path)
hftokenizer = AutoTokenizer.from_pretrained(hf_path)
hftokenizer.pad_token = hftokenizer.eos_token

# Tokenize the text prompt
input_ids = hftokenizer(PROMPT, return_tensors="pt", max_length=128, truncation=True).input_ids

# Run generation
out_ids = hfmodel.generate(input_ids=input_ids, max_new_tokens=15, do_sample=False)

# De-tokenize the generated response
response = hftokenizer.batch_decode(out_ids.numpy(), skip_special_tokens=True)[0][len(PROMPT):]
print("Response:", response)

## Next token generation

Now that we see that the model works, let's try to decompose its `hfmodel.generate` method because we will encounter it later. We should be able to get the same output as before, but by only using `forward` method of the model.

The code below is a simplified version of the actual loop you can find in the transformer's source code. It starts by initializing the current sequense to the given prompt and then generates 10 subsequent tokens - these tokens constitute the response of the model.

In [ ]:
from time import time
from transformers.generation.logits_process import LogitsProcessorList

logits_processor = LogitsProcessorList()

time_start = time()
current_seq = input_ids
N_TOKENS = 10
for idx in range(N_TOKENS):
    # Run model's `forward` on the current sequence.
    # 'logits' output would let us determine the next token for this sequence
    outputs = hfmodel(current_seq, return_dict=True).logits

    # Get the newly generated next token
    next_token_logits = outputs[:, -1, :]
    next_tokens_scores = logits_processor(current_seq, next_token_logits)
    next_tokens = torch.argmax(next_tokens_scores, dim=-1)

    print(hftokenizer.decode(next_tokens), end=' ', flush=True)

    # Append the new token to our sequence
    current_seq = torch.cat([current_seq, next_tokens[:, None]], dim=-1)

time_finish = time()
print(f"Prompt: {PROMPT}")
print("Response:", hftokenizer.batch_decode(current_seq.numpy(), skip_special_tokens=True)[0][len(PROMPT):])
print(f"Tokens per second: {N_TOKENS / (time_finish - time_start):.2f}\n")

## Convert to ONNX with optimum

Great! We were able to see the same response now with using only `forward` method of our model. We're now ready to use MAX Engine 🏎️ inference.
To do that, we start by getting an ONNX version of the model. The easiest way to do it is to use HuggingFace `optimum` tool which you can install as follows

In [ ]:
!python3 -m pip install -q optimum

Then the conversion to ONNX. This part can take a while. Also please make sure you've enough disk space.

In [ ]:
!optimum-cli export onnx --model "mistralai/Mistral-7B-v0.1" "./onnx/mistral-7b-onnx"

## Optional: Examine the ONNX model

**Caveat: 📢 if you want to run this 👇 part you will need around 95G of memory to be able to continue finishing the notebook. Otherwise, check out the included results below.**

In [ ]:
%%time
import onnx

# It will take a while to load in ONNX as well as occupying 28G more memory!
onnxmodel = onnx.load("./onnx/mistral-7b-onnx/model.onnx")

def print_dims(tensor):
    dims = []
    for dim in tensor.type.tensor_type.shape.dim:
        if dim.HasField("dim_value"):
            dims.append(str(dim.dim_value))
        elif dim.HasField("dim_param"):
            dims.append(str(dim.dim_param))
    print(onnx.TensorProto.DataType.Name(tensor.type.tensor_type.elem_type), end=" ")
    print("[", ", ".join(dims), "]")

print("=== Inputs ===")
for input_tensor in onnxmodel.graph.input:
    print(input_tensor.name, end=": ")
    print_dims(input_tensor)

print("\n=== Outputs ===")
for output_tensor in onnxmodel.graph.output:
    print(output_tensor.name, end=": ")
    print_dims(output_tensor)

```
=== Inputs ===
input_ids: INT64 [ batch_size, sequence_length ]
attention_mask: INT64 [ batch_size, past_sequence_length + 1 ]
position_ids: INT64 [ batch_size, sequence_length ]
past_key_values.0.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.0.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.1.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.1.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.2.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.2.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.3.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.3.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.4.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.4.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.5.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
past_key_values.5.value: FLOAT [ batch_size, 8, past_sequence_length, 128 ]

...

=== Outputs ===
logits: FLOAT [ batch_size, sequence_length, 32000 ]
present.0.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.0.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.1.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.1.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.2.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.2.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.3.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.3.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.4.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.4.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.5.key: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]
present.5.value: FLOAT [ batch_size, 8, past_sequence_length + 1, 128 ]

...
```

It might be quite surprising to see so many inputs and outputs in the model!

To decode what they all mean and how they should be used we will need to look into the [documentation of the MistralModel](https://huggingface.co/docs/transformers/v4.38.2/en/model_doc/mistral#transformers.MistralModel).

In short, we have the following inputs:
* input_ids
* position_ids
* attention_mask
* past_key_values

And the outputs will be:
* logits
* present_key_value

The `past_key_values.n.key` and `past_key_values.n.value` represents cached states from previous tokens in the sequence to avoid redundant recalculations, thus speeding up the generation process.

**Note** that since ONNX doesn't support dictionaries as a input/output type, the `key_value` is expanded into 32 pairs of individual tensors (32 is the number of attention layers). The number of KV heads is 8 which are part of its [Grouped Query Attention (GQA)](https://arxiv.org/abs/2305.13245) mechanism. The GQA mechanism allows the model to achieve faster inference times by optimizing the attention mechanism.

```
past_key_values.0.key: FLOAT [ batch_size, 8, past_sequence_length, 128 ]
```

Moreover, the size of each key/value vector is 128.

In order to use this model we will need to slightly modify our glue code to correctly weave all these values from each iteration to the next.
Specifically, we will need to pass the `key_values` from previous iteration to the current (for the first iteration they are initializes as empty tensors).
We will also need to correctly fill in `position_ids` and `attention_mask` tensors and update them on each iteration. We will not get into all the details of how exactly all these tensors affect the model behavior and should be used - this is an extremely interesting topic, but it is beyond the scope of this walkthrough.

## MAX Engine 🏎️

With that we're finally ready to use the MAX Engine 🏎️ for inference.

The code modifications follows our previous approach and is quite minimal. All we need to do is to load the ONNX model (whch can take a while) into an `InferenceSession` object and instead of using the `hfmodel` we will need to use `maxmodel.execute`, and pack the input values into a dictionary.

In [ ]:
%%time

from max import engine
# Create an InferenceSession and load the ONNX model
session = engine.InferenceSession()
maxmodel = session.load("./onnx/mistral-7b-onnx/model.onnx")

We can also quickly inspect the input and output metadata that match the ONNX version above

In [ ]:
for tensor in maxmodel.input_metadata:
    print(f'name: {tensor.name}, shape: {tensor.shape}, dtype: {tensor.dtype}')

for tensor in maxmodel.output_metadata:
    print(f'name: {tensor.name}, shape: {tensor.shape}, dtype: {tensor.dtype}')

Here is the how to get the response from our `maxmodel`. The token per second can be up to **2X** faster comparing to the PyTorch version. Note that this tutorial doesn't provide an accurate benchmark. For more, please check out our [performance dashboard](https://performance.modular.com/).

In [ ]:
inputs = {}
N_BATCH = 1
N_LAYERS = 32
N_HEADS = 8
KV_LEN = 128
# Initialize the additional layer to 0 for the first iteration:
for i in range(N_LAYERS):
    inputs[f"past_key_values.{i}.key"] = torch.zeros([N_BATCH, N_HEADS, 0, KV_LEN], dtype=torch.float).numpy()
    inputs[f"past_key_values.{i}.value"] = torch.zeros([N_BATCH, N_HEADS, 0, KV_LEN], dtype=torch.float).numpy()

current_seq = input_ids

time_start = time()
for idx in range(N_TOKENS):
    # Prepare inputs dictionary
    inputs["input_ids"] = current_seq.numpy()
    inputs["position_ids"] = torch.arange(inputs["input_ids"].shape[1], dtype=torch.long).unsqueeze(0).numpy()
    inputs["attention_mask"] = torch.ones([1, inputs[f"past_key_values.0.key"].shape[2] + inputs["input_ids"].shape[1]], dtype=torch.int64).numpy()

    # Run the model with MAX engine
    max_outputs = maxmodel.execute(**inputs)
    outputs = torch.from_numpy(max_outputs["logits"])

    # Get the newly generated next token
    next_token_logits = outputs[:, -1, :]
    next_tokens_scores = logits_processor(current_seq, next_token_logits)
    next_tokens = torch.argmax(next_tokens_scores, dim=-1)

    print(hftokenizer.decode(next_tokens), end=' ', flush=True)

    # Append the new token to our sequence
    current_seq = torch.cat([current_seq, next_tokens[:, None]], dim=-1)

    # Update the KV cache for the next iteration
    for i in range(N_LAYERS):
        inputs[f"past_key_values.{i}.key"] = max_outputs[f"present.{i}.key"]
        inputs[f"past_key_values.{i}.value"] = max_outputs[f"present.{i}.value"]

time_finish = time()

print(f"Prompt: {PROMPT}")
print("Response:", hftokenizer.batch_decode(current_seq.numpy(), skip_special_tokens=True)[0][len(PROMPT):])
print(f"Tokens per second: {idx/(time_finish-time_start):.2f}\n")

That is it! 🎉

Serving an LLM has historically been not an easy task, but hopefully this example lifts the curtain on how this can be done. MAX Engine 🏎️ doesn't (yet) make this process easier, however, if you've already gone this path with ONNX or TorchScript, switching to MAX should be trivial and bring easy performance wins.